In [184]:
import pandas as pd
import numpy as np

In [185]:
# Compute dist between atoms 1 & 2
# Inputs: coords_1 - atom 1 coords, coords_2 - atom 2 coords.
# Output: dist     - distance btwn atoms 1 & 2
def absolute_Distance(coords_1, coords_2):
    dist = np.sqrt((coords_2[0] - coords_1[0])**2 + (coords_2[1] - coords_1[1])**2 + (coords_2[2] - coords_1[2])**2)
    return dist

**Specify the cluster size: N**

Here, specify the size, N, of the LJ cluster you want to analyze.
Download the corresponding file, which contains the xyz coordinates of
each atom in this size-N cluster.

In [186]:
import requests

natom = input("Specify cluster size: ")
natom = int(natom)

# N > 151 or N < 2 are inaccessible
while natom > 150 or natom < 3:
    N = input("Specify a cluster size less than or equal to 150 and greater than or equal to 3: ") 
    natom = int(natom)

# URL of the file you want to download
url = f"http://doye.chem.ox.ac.uk/jon/structures/LJ/points/{natom}"

# send a GET request to the URL
response = requests.get(url)

# specify the path where the file will be saved
file_name = f"LJ_cluster_size_{N}.txt"
file_path = f"C:\\Users\\mrmuf\\OneDrive\\Desktop\\chem193\\{file_name}"

# save the file
with open(file_path, "wb") as file:
    file.write(response.content)

**Create a list of every atom's xyz coordinates**

Next, we'll clean the .txt of whitespace, and format it for use as a dataframe object - we'll store its contents in a 'raw list' of unformatted coordinates.

Then, we'll segment every 3 coordinates into lists, which each represent an atom - every element will represent one atom's x-, y-, and z-coordinates.

In [187]:
### Implementation - scraping coords into a list ###
list_of_atom_coords = [] # raw list of unsegmented atom coords
with open(file_path) as file:
    text   = file.read()
    coords = text.split()
    for coord in coords:
        list_of_atom_coords.append(coord)
    # print(list_of_atom_coords)

formatted_coords = [] # formatted list of coords ~ each atom
for coord in list_of_atom_coords:
    coords_len = len(list_of_atom_coords)
    formatted_coords = [list_of_atom_coords[i: i + 3] for i in range(0, coords_len, 3)] # split every 3 coords into a list for one atom
print(formatted_coords)

# convert all coords to float for all N atoms (N = len(formatted_coords))
for i in range(len(formatted_coords)):
    formatted_coords[i] = list(map(float, formatted_coords[i]))

[['1.0058004314', '1.2511810451', '1.4482229409'], ['-2.1342612054', '-0.1926609572', '0.2867913287'], ['1.0253477798', '-1.8410416964', '-0.4834377754'], ['-1.0168005815', '-1.3465953712', '-1.2892324948'], ['2.1174255262', '0.0945645651', '-0.1299602145'], ['-1.0363299363', '1.7398807150', '0.6388898149'], ['0.7642007096', '-0.1264727253', '-1.9088788603'], ['-1.1816378542', '0.8886834992', '-1.4345163230'], ['0.7521498880', '1.7778652736', '-0.7192558114'], ['-0.7848687999', '-0.9386649200', '1.4946425496'], ['0.1995997470', '-0.4859932920', '1.8587730437'], ['0.2057313636', '-1.4554616805', '1.2531509762'], ['1.1105363622', '-1.4304380819', '0.5607213363'], ['0.1311852250', '-1.8807569798', '0.1984837336'], ['1.1006483641', '0.1335398121', '1.5377065525'], ['-1.4668715418', '-1.0470398740', '0.5880443279'], ['0.1152084013', '0.6476370887', '1.7779405908'], ['-1.4729661324', '-0.0826263789', '1.1905138743'], ['-0.9169385245', '0.8935621333', '1.3682087402'], ['-0.9009827007', '-1.62

**Compute Relative Distances**

Then, we'll use absolute_Distance() to compute the relative distances between each atom in the cluster; and store these distances in a list. The matrix storing the pairwise interaction matrix is initialized as all zeros. 

In [188]:
from itertools import combinations

distances = []
pairwiseInt = np.zeros((natom, natom))
# Generate all pairs of atoms & calculate relative distances
for (atom1, atom2) in combinations(formatted_coords, 2):
    dist = absolute_Distance(atom1, atom2)
    distances.append(float(dist))
    
print(distances)

[3.6460376027032773, 3.6460302665743023, 4.281707536592388, 2.250363105186583, 2.2503652847668616, 3.6368163055468523, 3.6368222925684592, 2.2449273966856302, 2.829147650240465, 1.9586438185425505, 2.829148004029778, 2.82660748171025, 3.4836526286849354, 1.1252223659278213, 3.4836522969866315, 1.1252258440154064, 2.8266127936344976, 1.9573499050824046, 3.8313375800206155, 1.9573417530748147, 3.826823082928088, 1.1252727692468352, 3.8268162988946792, 2.821347172849078, 1.1252722557286685, 2.8213527931156945, 3.474053653783864, 1.1223682761206266, 3.4740387736974734, 3.8219459177935984, 2.8160064962276383, 1.9508042729814605, 2.816015719472759, 1.9508040949696077, 3.8219487635337988, 2.813615458876039, 3.469711009293042, 2.8136166591736527, 2.139580529146017, 1.921643677522115, 1.9216380425928392, 2.78746205107532, 2.776644514718239, 1.0901706730746155, 2.7766352082542474, 3.191200249731286, 1.9161720714341344, 1.9161776982849714, 2.77030265727282, 1.913048437679703, 2.7703022531728103, 

**Lennard Jones PE**

This function will be used to calculate the Lennard Jones PE between atoms 1 and 2. 

In [189]:
# Inputs: x1 and x2 are lists that contains the x, y, and z coordinates of an atom
def lennard_Jones_PE(atom1, atom2):
    r = absolute_Distance(atom1, atom2)
    U = 4*((1/r)**12 - (1/r)**6)
    return U

**Update Pairwise Interaction Matrix**

This function can be used to calculate the pairwise interaction between each atom

In [190]:
def update_pairwiseInt(natom, coords):
    for i in range(natom):
        # Grab the row'th atom in the position matrix
        atom1 = coords[i]
        for j in range(i, natom):
            # Check if we are calculating U_ii. If we are, leave it as 0
            if i == j:
                pass
            else:
                atom2 = coords[j]
                pairwiseInt[i][j] = lennard_Jones_PE(atom1, atom2)
                pairwiseInt[j][i] = lennard_Jones_PE(atom1, atom2)
    return 

In [191]:
def tot_Energy(natom, pairwiseInt):
    tot_Energy = 0
    for i in range(natom):
        for j in range(i, natom):
            tot_Energy += pairwiseInt[i][j]
    return tot_Energy

In [192]:
update_pairwiseInt(natom, formatted_coords)
tot_Energy = tot_Energy(natom, pairwiseInt)
print("The total energy of this system is", round(tot_Energy))

The total energy of this system is -258
